In [135]:
import pickle

with open(r'C:\Users\IT\Documents\UCL\pyssem\scenario-properties-baseline.pkl', 'rb') as f:
    scen_properties = pickle.load(f)

# with open(r'C:\Users\IT\Documents\UCL\pyssem\scenario-properties-baseline.pkl', 'rb') as f:
#     baseline = pickle.load(f)

In [124]:
import re

debris_species_names = [species.sym_name for species in scen_properties.species['debris']]
pattern = re.compile(r'^N_[^_]+kg')
unique_debris_names = set()

for name in debris_species_names:
    match = pattern.match(name)
    if match:
        unique_debris_names.add(match.group())
unique_debris_names = list(unique_debris_names)

mass_bins = unique_debris_names
ecc_bins = scen_properties.species['debris'][0].eccentricity_bins

all_elliptical_collision_species = scen_properties.collision_pairs

In [133]:
import numpy as np
from sympy import pi, S
from tqdm import tqdm

def main():
    n_shells = scen_properties.n_shells

    # Initialize debris species lists
    debris_species_names = []

    for s in range(n_shells):
        for mass_value in mass_bins:
            for ecc_value in ecc_bins:
                debris_species_name = f'{mass_value}_e{ecc_value}_{s + 1}'
                debris_species_names.append(debris_species_name)

    # Ensure that debris species variables are included in 'all_symbolic_vars'
    all_symbolic_vars = scen_properties.all_symbolic_vars

    # Rebuild the mappings to include the debris species
    species_name_to_idx = {str(var): idx for idx, var in enumerate(all_symbolic_vars)}

    # Switch to a NumPy matrix for performance
    n_species = len(all_symbolic_vars)
    equations = np.zeros((n_shells, n_species))

    for collision_pair in tqdm(all_elliptical_collision_species, total=len(all_elliptical_collision_species), desc="Generating collision equations"):
        
        if len(collision_pair.collision_pair_by_shell) == 0:
            # If no fragments, skip this collision pair
            continue

        species1 = collision_pair.species1
        species2 = collision_pair.species2
        species1_name = species1.sym_name
        species2_name = species2.sym_name

        # Conversion factor
        meter_to_km = 1 / 1000

        # Compute collision cross-section (sigma)
        sigma = (species1.radius * meter_to_km + species2.radius * meter_to_km) ** 2

        # Compute collision rate (phi) for each shell
        phi_matrix = pi * scen_properties.v_imp2 / (scen_properties.V * meter_to_km**3) * sigma * 86400 * 365.25
        phi_matrix = np.atleast_1d(phi_matrix)  # Ensure it's at least 1D

        for elliptical_pair in collision_pair.collision_pair_by_shell:
            s_source = elliptical_pair.shell_index  # Source shell index

            # Get symbolic variable names for species 1 and 2
            species1_var_name = f'{species1_name}_{s_source + 1}'
            species2_var_name = f'{species2_name}_{s_source + 1}'

            # Retrieve index for each species
            idx_species1 = species_name_to_idx.get(species1_var_name)
            idx_species2 = species_name_to_idx.get(species2_var_name)

            if idx_species1 is None or idx_species2 is None:
                continue  # Skip if species variables are not found

            phi_s = phi_matrix[s_source]

            # Process fragments from the collision
            fragments = elliptical_pair.fragments  # Shape [n_destination_shells, n_mass_bins, n_ecc_bins]

            if fragments is not None:
                n_destination_shells, n_mass_bins, n_ecc_bins = fragments.shape

                for s_destination in range(n_destination_shells):
                    fragments_sd = fragments[s_destination]  # Fragments in destination shell
                    non_zero_indices = np.nonzero(fragments_sd)  # Only process non-zero fragments

                    for mass_bin_index, ecc_bin_index in zip(*non_zero_indices):
                        num_frags = fragments_sd[mass_bin_index, ecc_bin_index]
                        
                        mass_value = mass_bins[mass_bin_index]
                        ecc_value = ecc_bins[ecc_bin_index]
                        # Generate debris species variable name
                        debris_species_name = f'{mass_value}_e{ecc_value}_{s_destination + 1}'

                        idx_debris = species_name_to_idx.get(debris_species_name)
                        if idx_debris is not None:
                            # Use NumPy array to update matrix
                            delta_gain = phi_s * equations[s_source, idx_species1] * equations[s_source, idx_species2] * num_frags
                            equations[s_destination, idx_debris] += delta_gain
        
        # Now, extract the equations for the debris species and store them in a (10, 20) matrix
        # Initialize the debris equations matrix
        debris_length = len(mass_bins) * len(ecc_bins) 
        equations_debris = np.zeros((n_shells, debris_length))

        # Map debris species variable names to indices within the shell
        debris_species_idx_within_shell = {}
        for idx_within_shell, debris_species_name in enumerate(debris_species_names[:debris_length]):
            base_name = '_'.join(debris_species_name.split('_')[:-1])
            debris_species_idx_within_shell[base_name] = idx_within_shell

        # Populate the debris equations matrix
        for s in range(n_shells):
            for mass_value in mass_bins:
                for ecc_value in ecc_bins:
                    debris_species_name = f'{mass_value}_e{ecc_value}_{s + 1}'
                    base_name = f'{mass_value}_e{ecc_value}'
                    idx_species = species_name_to_idx.get(debris_species_name)
                    idx_debris = debris_species_idx_within_shell.get(base_name)
                    if idx_species is not None and idx_debris is not None:
                        eq = equations[s, idx_species]
                        equations_debris[s, idx_debris] += eq

        species_names = scen_properties.species_names
        species1_idx = species_names.index(species1.sym_name)
        species2_idx = species_names.index(species2.sym_name)
        
        # Find the start of the debris species index, which will be the first item in species_names that starts with 'N_'
        debris_start_idx = next(i for i, name in enumerate(species_names) if name.startswith('N_'))

        try:
            eq_s1 = collision_pair.gamma[:, 1] * phi_matrix[:, None] * equations[:, species1_idx] * equations[:, species2_idx]
            eq_s2 = eq_s1.copy()  # Similar logic for eq_s2
        except Exception as e:
            print(f"Exception caught: {e}")
            print("Error in multiplying gammas, check that each component is a column vector and correct shape.")
            
        eqs = np.zeros((n_shells, scen_properties.species_length))

        # Add in eq_1 at species1_idx and eq_2 at species2_idx
        eqs[:, species1_idx] = eq_s1
        eqs[:, species2_idx] += eq_s2

        # Loop through each debris species
        for i in range(len(scen_properties.species['debris'])):
            # Calculate the corresponding index in the overall species list
            deb_index = debris_start_idx + i
            # Assign the columns from equations_debris to the appropriate columns in eqs
            eqs[:, deb_index] = equations_debris[:, i]

        collision_pair.eqs = eqs


In [134]:
import cProfile

cProfile.run('main()')

Generating collision equations:   0%|          | 0/119 [00:00<?, ?it/s]

Exception caught: Matrix size mismatch: (20, 1) * (20, 1).
Error in multiplying gammas, check that each component is a column vector and correct shape.
         133048 function calls (131630 primitive calls) in 0.094 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 755517121.py:116(<genexpr>)
        1    0.000    0.000    0.002    0.002 755517121.py:21(<dictcomp>)
        1    0.008    0.008    0.094    0.094 755517121.py:5(main)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_1d)
      400    0.000    0.000    0.002    0.000 <__array_function__ internals>:177(nonzero)
      314    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(reshape)
      314    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(size)
       19    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1033(_handle_fromlist

UnboundLocalError: local variable 'eq_s1' referenced before assignment